## Challenges ##
1. Merged data splitted into different lines, i.e.
|column 1|column 2|
|-|-|
|Maybank|Maybank|
|Islami|Investment|
|Berhad|Bank Berhad||
- which are actually "Maybank Islamic Berhad" and "Maybank Investment Bank Berhad"

2. Different number of set in a column, i.e

|for|example|
|-|-|
|2022 2021|...|
|Note RM’000 RM’000|...|
|5 37,573,869 41,483,9263|...| 

3. Should be blank for first row first column, i.e.



|RM’ million|%||
|-|-|-|
|Interest...|||
|Loans...|541,888|4.58|
|financial...|57,521|1.83|

In [1]:
# function 1 - replace spaces that meet critirea with pipeline
import re

def space_to_pipeline(text):

    parts = []
    if isinstance(text, str):
        
        # Skip format
        formats = [
            r'.*\s?\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec).*\s?',  # Matches "FY 31 Dec 2021" format
            r'.*\s?\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December).*\s?', # Matches "31 December" format
            r'.*\s?Tier [12].*\s?',
            r'.*\s?Stage [123].*\s?',
            r'.*\s?12-month.*\s?',
            r'^.*[a-zA-Z]+\s+\d+%?\s+[a-zA-Z]+.*$', # numbers within sentence
            r'.*[a-zA-Z\s]+(\d+)\([a-zA-Z]+\).*'  # bullet point format
        ]
        
        pattern = '|'.join(formats)
        
        if re.match(pattern, text):
            return text  # Skip processing for date format cells

        else:
        
            # Replace spaces between numbers with a pipeline, i.e. "123 789" to "123|789" [no 2nd group to consider overlaping]
            # text = re.sub(r'(\d+)\s+(\d+)', r'\1|\2', text)
            text = re.sub(r'(\d+)\s+(?=\d)', r'\1|', text)
            
            # Replace spaces between alphabet and number with a pipeline, i.e. "abc 123" to "abc|123"
            text = re.sub(r'([a-zA-Z]+)\s+(\d+)', r'\1|\2', text)

            # Replace spaces between ")" and "(" with a pipeline, i.e. ") (" to ")|("
            text = re.sub(r'\)\s+\(', r')|(', text)
            
            # Replace spaces between number and "(" with a pipeline, i.e. "123 (" to "123|("
            text = re.sub(r'(\d+)\s+\(', r'\1|(', text)
            
            # Replace spaces between "(" and number with a pipeline, i.e. ") 456" to ")|456"
            text = re.sub(r'\)\s+(\d+)', r')|\1', text)
    
            # Replace spaces between number, space, and punctuation < or > with a pipeline, i.e. "123 >" to "123|>"
            # text = re.sub(r'(\d+)\s+([<>])\s+(\d+)', r'\1|\2\3', text)
            text = re.sub(r'(\d+)\s+([<>])\s*(\d+)', r'\1|\2\3', text)
    
            # Replace spaces between number and space and alphabet FY with a pipeline, i.e. "123 FY" to "123|FY"
            text = re.sub(r'(\d+)\s+(FY)', r'\1|\2', text)
    
            # Replace spaces between number and space and "variance", ignore case sensitive, with a pipeline, i.e. "123 variance" to "123|variance"
            text = re.sub(r'(\d+)\s+(?i:variance)', r'\1|Variance', text)

            # Replace spaces between number and dash (-, which indicating zero amount) with a pipeline, i.e. "123 -" to "123|-"
            text = re.sub(r'(\d+)\s+–', r'\1|–', text)

            # Replace spaces between dash (-, which indicating zero amount) and numbers with a pipeline, i.e. "- 789" to "-|789"
            text = re.sub(r'–\s+(\d+)', r'–|\1', text)

            # Replace spaces between dash and dash with a pipeline, i.e. "- -" to "-|-"
            text = re.sub(r'–\s+–', r'–|–', text)
            
            # Replace spaces between dash and number with a pipeline, i.e. "- 123" to "-|123"
            text = re.sub(r'–\s+(\d+)', r'–|\1', text)
            
            # Replace spaces between number and dash with a pipeline, i.e. "123 -" to "123|-"
            text = re.sub(r'(\d+)–', r'\1|–', text)
            
            # Replace spaces between dash and "(" with a pipeline, i.e. "- (" to "-|("
            text = re.sub(r'–\s+\(', r'–|(', text)
            
            # Replace spaces between ")" and dash with a pipeline, i.e. ") -" to ")|-"
            text = re.sub(r'\)\s+–', r')|–', text)
            
            # Replace spaces before and after ")" followed by a number with a pipeline, i.e. ") 456" to ")|456"
            text = re.sub(r'\)\s+(\d+)', r')|\1', text)
    
            # Replace spaces between alphabet and space and "%", with a pipeline, i.e. "abc %" to "abc|%"
            text = re.sub(r'([a-zA-Z]+)\s+%', r'\1|%', text)
    
            # Replace spaces between % and space and alphabet, ignore case sensitive, with a pipeline, i.e. "% abc" to "%|abc"
            text = re.sub(r'%\s+(?i:[a-zA-Z]+)', r'%|Change', text)

            # Replace spaces between % and number with a pipeline, i.e. "14.100% 27.260%" to "14.100%|27.260%"
            text = re.sub(r'%\s+(\d+)', r'%|\1', text)

            # Replace spaces between RM’000 with a pipeline, i.e. "Note RM’000" to "Note|RM’000"
            text = re.sub(r'([a-zA-Z\d]+)\s+RM’000', r'\1|RM’000', text)

            # Replace spaces between RM’000 with a pipeline, i.e. "RM’000 RM’000" to "RM’000|RM’000"
            text = re.sub(r'RM’000\s+RM’000', r'RM’000|RM’000', text)

            # Replace spaces between RM’123 with a pipeline, i.e. "RM’123 RM’789" to "RM’123|RM’789"
            text = re.sub(r'(\d+)\s+(RM)', r'\1|\2', text)

            # Replace spaces between Group and Bank with a pipeline, i.e. "Group Bank" to "Group|Bank"
            text = re.sub(r'([Gg]roup)\s+([Bb]ank)', r'\1|\2', text)
            
            # Replace spaces between Maybank with a pipeline, i.e. "Maybank Maybank" to "Maybank|Maybank"
            text = re.sub(r'([Mm]aybank)\s+([Mm]aybank)', r'\1|\2', text)

            # Replace spaces between Islamic and Investment with a pipeline, i.e. "Islamic Investment" to "Islamic|Investment"
            text = re.sub(r'([Ii]slamic)\s+([Ii]nvestment)', r'\1|\2', text)

            # Replace spaces between Berhad and Bank with a pipeline, i.e. "Berhad Bank" to "Berhad|Bank"
            text = re.sub(r'([Bb]erhad)\s+([Bb]ank)', r'\1|\2', text)
        
    return text

In [2]:
# function 2 - count the maximum number of pipiline in a column (for inserting new columns)
def max_pipeline_count(column_data):
    max_count = 0
    for cell_text in column_data:
        if isinstance(cell_text, str):
            pipeline_count = cell_text.count('|')
            if pipeline_count > max_count:
                max_count = pipeline_count
    return max_count

In [3]:
# function 3 - read in an Excel file, exercise above functions, then write to a new Excel file
import pandas as pd
def custom_split_excel(input_file, output_file):
    # Read the Excel file
    xls = pd.ExcelFile(input_file)
    
    with pd.ExcelWriter(output_file) as writer:
        # Iterate over each sheet in the Excel file
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Iterate over each column with data in the DataFrame
            for col in df.columns:
                # function 1 - to replace the spaces with pipelines
                df[col] = df[col].apply(space_to_pipeline)

                # function 2 - create new columns
                max_count = max_pipeline_count(df[col])
                if max_count > 0:
                    for i in range(max_count+1):
                        new_col_name = f"{col}_{i+1}"
                        df.insert(df.columns.get_loc(col) + i + 1, new_col_name, "")

                        # split by pipeline
                        df[new_col_name] = df[col].str.split('|', expand=True)[i]

                    # remove initial column
                    df.drop(col, axis=1, inplace=True)
            
            # Write the modified DataFrame back to the new Excel file
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [4]:
custom_split_excel(input_file="path\to\original\excel",
                   output_file="path\to\new\excel")